# Using LangChain with GPT4All and Custom Prompts

## References

1. [LangChain docs on using the GPT4All library](https://python.langchain.com/docs/integrations/llms/gpt4all)

## Code

### Import libraries

In [1]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### Setup

In [2]:
local_path = r"C:\Users\deshi\Code\gpt4all-models\mistral-7b-instruct-v0.1.Q4_0.gguf"
# local_path = r"C:\Users\deshi\Code\gpt4all-models\orca-mini-3b-gguf2-q4_0.gguf"

In [3]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(
    model=local_path, 
    callbacks=callbacks, 
    verbose=True,
    max_tokens=512,
    n_predict=512,
    n_batch=4
)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

SyntaxError: invalid syntax (1093496073.py, line 11)

### Standard template

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llmChain = LLMChain(prompt=prompt, llm=llm)

### Regular Inference

In [ ]:
question = """
Write a program to calculate the nth fibonacci number given a user input integer n. Also, print out all the n fibonacci numbers leading upto n in a list. Format the output in markdown format so it is easy to export to a markdown document.
"""

llmChain.run(question)

### Function Documentation Inference with Standard Template

In [ ]:
testFunction = """
n = int(input("Enter an integer: "))
result = 0
previous_two = 1
while result < n:
    next_two = previous_two + previous_two
    result += next_two
    previous_two = next_two
    print("The nth Fibonacci number is:", result)
"""

question = f"""
Here's my function in Python:\

{testFunction}\

Given the definition of a function in any programming language (particularly Python and C++), please generate it's stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.
"""

In [ ]:
llmChain.run(question)

### Custom Prompt Template

In [ ]:
template = """
Given the definition of a function in any programming language, please generate it's documentation. The documentation needs to have fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line.

This is a function in {language}:
{function}

The response needs to be structured as:

# Function:
The function definition and body here.
# Arguments:
List of arguments along with their types.
# Return values:
List of all the values returned by the function with their types.
# Explanation:
Line-by-line explanation of the function's logic and implementation.
# Use cases:
How the function may be used in an intended context.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["language", "function"],
)

In [ ]:
llmChain = LLMChain(
    prompt=prompt, 
    llm=llm,
)

In [ ]:
# Be specific about the language to avoid LLM hallucination
language = "Python"

print(prompt.format(language=language, function=testFunction))

In [ ]:
llmChain.run({"language": language, "function": testFunction})